# blah blah

In [1]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [96]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-28 16:14:24 [INFO] Started loading binary file
2019-10-28 16:14:24 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-28 16:14:24 [INFO] Finished loading binary file in 0.04s (0.06s CPU)


In [97]:
desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
index,int64,0/32237,32237
Endkunde_NR,int64,0/32237,32237
Endkunde,object,0/32237,30291
EK_Aktiv,int64,0/32237,2
EK_Kam_Betreut,int64,0/32237,2
EK_Land,object,49/32188,45
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
Agentur,object,20999/11238,3247
EK_BG,object,1829/30408,83


# CRM STUFF

In [69]:
crm_data = load_bin("vkprog\\crm_data.feather").astype({"ENDKUNDE_NR": "int64"})

2019-10-28 14:34:23 [INFO] Started loading binary file
2019-10-28 14:34:23 [INFO] Reading from file C:\Users\stc\data\vkprog\crm_data.feather
2019-10-28 14:34:24 [INFO] Finished loading binary file in 0.21s (0.25s CPU)


In [71]:
display(crm_data.sample(5, random_state=42))
desc_col(crm_data)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
490351,Traffic SR-Actions 2017 - Changement d'adresse,E-Mail,2017-12-22,Marketing,Cédric Genoud (TRAFFIC ROMANDIE),CGE,486920,NaT,NaT,NaN
937554,Telemarketing,Besuch,2015-06-23,Verkauf,Massimo Bonotto (REGION OST CHUR),BON,604538,NaT,NaT,NaN
27740,2013_Weihnachtskarten - Mitte_2013,Brief,2013-11-22,Marketing,Maja Loosli (REGION VERKAUF MITTE),MAL,103253,NaT,NaT,NaN
799222,Ouest - Aktionen 2019 - Promotion d'été 2019 -...,E-Mail,2019-05-23,Marketing,Lisa Conte (REGION WEST GENF),LIC,570321,NaT,NaT,NaN
915179,Anfrage Buswerbung,E-Mail,2015-01-13,Verkauf,Domenico Torre (APG WINTERTHUR),DTO,598472,NaT,NaT,NaN


,DTYPE,NULLS,UNIQUE
BETREFF,category,26/1056374,336940
KANAL,category,0/1056400,12
STARTTERMIN,datetime64[ns],0/1056400,3817
QUELLE,category,0/1056400,3
VERANTWORTLICH,category,0/1056400,337
KUERZEL,category,1/1056399,340
ENDKUNDE_NR,int64,0/1056400,51555
VB_FILTER_VON,datetime64[ns],1054692/1708,13
VB_FILTER_BIS,datetime64[ns],1055145/1255,37
VB_FILTER_GRUND,category,1054709/1691,60


I need:
- ``Letzte_VBs``
- ``KZ_letzter_Ktkt`` (Letzter VB-Kontakt, Kürzel)
- ``Kanal`` (Kontaktweg: Email, Telefon, Brieftaube...)
- ``Betreff`` (Inhalt des letzten Kontakts)

In [6]:
crm_data.sort_values("STARTTERMIN", ascending=False).head(5)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
624924,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2024-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
69410,A contacter,Besuch,2023-10-21,Verkauf,Michel Zbinden (REGION WEST GENF),MZB,109933,NaT,NaT,NaN
624923,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2023-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624922,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2022-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624921,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2021-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN


In [81]:
def collect(s, sep=","):
    return sep.join(map(str, s[s.notna()].unique()))

In [38]:
def last_notna(s):
        try:
            return s.loc[s.notnull()].iat[-1]
        except IndexError:
            return np.NaN

In [93]:
def crm_info():
    today = pd.Timestamp.now()

    ## Information about last contact: Kanal, VB, Betreff
    row_select = (crm_data.loc[:,"STARTTERMIN"] <= today)
    container_crm = (crm_data.loc[row_select,:]
                             .sort_values(["ENDKUNDE_NR","STARTTERMIN"])
                             .groupby("ENDKUNDE_NR", as_index=False)
                             .agg(
                                 {"KUERZEL": last_notna,
                                  "KANAL": last_notna,
                                  "BETREFF": last_notna,
                                  #"STARTTERMIN": last_notna,
                                 }
                             )
                             .rename(columns={
                                 "KUERZEL": "Letzter_Kontakt",
                                 "KANAL": "Kanal",
                                 "BETREFF": "Betreff",
                                 "ENDKUNDE_NR": "Endkunde_NR",
                             })
                    )

    # Letzte CRM-Kontakte (alle der letzten zwei Kalendarjahre)
    row_select = (crm_data.loc[:,"STARTTERMIN"]
                          .apply(lambda x: 2 >= today.isocalendar()[0] - x.isocalendar()[0])
                  & (crm_data.loc[:,"STARTTERMIN"] <= today)
                 )

    crm_letzte_vbs = (crm_data.loc[row_select,:]
                              .groupby("ENDKUNDE_NR", as_index=False)
                              .agg(
                                  {"KUERZEL": collect,
                                  }
                              )
                              .rename(columns={
                                  "KUERZEL": "Letzte_CRM_Ktkts",
                                  "ENDKUNDE_NR": "Endkunde_NR",})
                     )
    # Combine
    container_crm = pd.merge(container_crm, crm_letzte_vbs, on="Endkunde_NR",how="left")

    return container_crm

In [94]:
ek_crm_info = crm_info()

# BD stuff

In [99]:
bookings_raw = load_bin("vkprog\\bd_data.feather").rename(
            mapper=lambda name: cap_words(name, sep="_"), axis="columns"
        )
booking_raw = bookings_raw.loc[(bookings_raw.Netto > 0)].pipe(clean_up_categoricals)

2019-10-28 16:44:12 [INFO] Started loading binary file
2019-10-28 16:44:12 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-28 16:44:12 [INFO] Finished loading binary file in 0.16s (0.73s CPU)


In [100]:
desc_col(booking_raw)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/680311,32237
Endkunde,category,0/680311,30291
EK_Abc,category,1155/679156,8
EK_Boni,category,76/680235,3
EK_Plz,category,165/680146,2968
EK_Ort,category,165/680146,3019
EK_Land,category,165/680146,45
EK_HB_Apg_Kurzz,category,28683/651628,91
EK_Kam_Betreut,int64,0/680311,2
EK_Aktiv,int64,0/680311,2


In [101]:
booking_raw.head(5)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
2,123010,Perrodin Voyages,NaN,gut,1920,Martigny,SCHWEIZ,NaN,0,1,NaN,NaN,VRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035675,Traffic,540946,KDVTLG,4,2008-10-01,2009-01-01,Traffic Auftrag,2008-10-01,NaT,2009-01-01,365,Ja,1067,1067,NaN,Hecktafel,2009,1,1,1,2008,40,39,37
3,411700,"Maharaja, traiteur, cuisine indienne",E,gut,1004,Lausanne,SCHWEIZ,SAO,0,1,NaN,NaN,YTA,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035902,Traffic,541110,KDVTLG,4,2008-10-01,2009-01-01,Traffic Auftrag,2008-10-01,NaT,2009-01-01,365,Ja,1125,1125,NaN,"Schiebekarton,Seitenbanner,Teilheck Fenster",2009,1,1,1,2008,40,39,37
4,482093,McDonald's Suisse Restaurants Sàrl,A,gut,1023,Crissier,SCHWEIZ,BEG,1,1,OMD Schweiz AG,FER,NIE,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035961,APG|SGA,541161,KDVTLG,4,2008-10-01,2008-12-29,langfristiger Vertrag,2008-10-01,NaT,2008-12-29,371,Ja,5500,5500,"90768,91372,93379,A0230,A0261,A0351,A0371,A058...","Dachlängsformat,Diverse,F12,F12L,F12LR,F200,F2...",2009,1,1,1,2008,40,39,37
15,503170,Fondazione agli Sassu e Helenita Oliveres,E,gut,6900,Lugano,SCHWEIZ,SOL,0,1,NaN,NaN,ROS,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,1041421,APG|SGA,542512,KPG,4,2008-10-13,2009-02-02,Kommerziell,2008-10-13,NaT,2009-02-02,14,Nein,3675,3675,"A5002,A5113,A5192,A5250",F12,2009,6,5,5,2008,42,41,41
16,122839,Opéra de Lausanne,B,gut,1002,Lausanne,SCHWEIZ,MZB,0,1,NaN,NaN,BTU,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,1058702,APG|SGA,543250,KPG,4,2008-10-16,2009-01-26,Kommerziell,2008-11-19,NaT,2009-01-26,14,Nein,860,860,"95401,95822,A0351,A2125,A2196,A5586,A5890,A593...","F12,F12L,F200,F200L,F4C,F4M,F4S,TrafficMediaSc...",2009,5,5,5,2008,42,41,41


In [165]:
def booking_nettos_vbs(booking_raw):
    today = pd.Timestamp.now()
    booking_raw = booking_raw.astype({"Kamp_Erfass_Jahr": "int64"})

    ##
    row_select = ((booking_raw.loc[:,"Vertrag"] == "Ja") &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] <= today.isocalendar()[0]) &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] >= (today.isocalendar()[0])-4) 
                 )

    booking_nettos = pd.pivot_table(
        booking_raw.loc[row_select,:],
        values  = ["Netto"],
        aggfunc = {"Netto": np.nansum},
        columns = ["Kamp_Erfass_Jahr"],
        index   = ["Endkunde_NR"],
        fill_value= 0
        )

    booking_nettos_flattened = pd.DataFrame(booking_nettos.to_records(index=False))
    booking_nettos_flattened.loc[:,"Endkunde_NR"] = pd.Series(booking_nettos.index)
    col_names = ["Net_"+str(x) for x in range(today.isocalendar()[0]-4, today.isocalendar()[0]+1)] + ["Endkunde_NR"]
    booking_nettos_flattened.columns = col_names

    ##
    row_select = ((booking_raw.loc[:,"Vertrag"] == "Ja") &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] <= today.isocalendar()[0]) &
                  (booking_raw.loc[:,"Kamp_Erfass_Jahr"] >= (today.isocalendar()[0])-2) 
                 )

    booking_letzte_vbs = (booking_raw
        .loc[row_select,:]
        .groupby("Endkunde_NR", as_index=False)
        .agg({"Verkaufsberater": collect})
        .rename({"Verkaufsberater": "Letzte_VBs"})
        )

    ## merge
    ek_booking = pd.merge(
        booking_nettos_flattened,
        booking_letzte_vbs,
        on="Endkunde_NR",
        how="left")
    return ek_booking

In [166]:
ek_booking = booking_nettos_vbs(booking_raw=booking_raw)

In [168]:
ek_booking.sample(5,random_state=42)

,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Endkunde_NR,Verkaufsberater
1261,18815,26815,18815,21425,21425,137604,MBU
2191,12500,0,0,0,0,478047,NaN
3370,0,0,0,5000,5192,552438,CGA
805,3500,3500,3500,3500,3500,124025,"REM,DTO"
2977,1500,1500,1500,1500,0,518697,DTO


In [158]:
booking_nettos_flattened.sample(5,random_state=42)

,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Endkunde_NR
1261,18815,26815,18815,21425,21425,137604
2191,12500,0,0,0,0,478047
3370,0,0,0,5000,5192,552438
805,3500,3500,3500,3500,3500,124025
2977,1500,1500,1500,1500,0,518697


['Net_2015', 'Net_2016', 'Net_2017', 'Net_2018', 'Net_2019', 'Endkunde_NR']

In [152]:
str(2019)

'2019'